# Simonのアルゴリズム
f(x) = f(x xor s) となる周期 s を求める

入力を n bit とすると、O(n^3)

In [1]:
from blueqat import Circuit

## f(x) を出力するオラクル
n = 2, 周期 s = 1, 2, 3

In [2]:
# s = 1
def n2s1(c):
    return c.cx[0, 3]

# s = 2
def n2s2(c):
    return c.cx[1, 2].x[1].cx[1, 3].x[1]

# s = 3
def n2s3(c):
    return c.ccx[0, 1, 2].ccx[0, 1, 3].x[:2].ccx[0, 1, 2].ccx[0, 1, 3].x[:2]

n = 3, s = 5

    000: 101
    001: 001
    010: 100
    011: 000
    100: 001
    101: 101
    110: 000
    111: 100

In [3]:
def n3s3(c):
    return c.x[1].cx[1,5].x[1:3].ccx[0,2,3].x[0,2].ccx[0,2,3]

### 各オラクルの出力

In [4]:
n2s1(Circuit(4).h[:2]).m[:].run(shots=100)

Counter({'0000': 27, '1001': 22, '0100': 23, '1101': 28})

In [5]:
n2s2(Circuit(4).h[:2]).m[:].run(shots=100)

Counter({'0110': 22, '1110': 31, '0001': 22, '1001': 25})

In [6]:
n2s3(Circuit(4).h[:2]).m[:].run(shots=100)

Counter({'0011': 34, '1111': 27, '0100': 26, '1000': 13})

In [7]:
n3s3(Circuit(6).h[:3]).m[:].run(shots=100)

Counter({'010100': 11,
         '110000': 15,
         '101101': 13,
         '100001': 9,
         '001001': 14,
         '111100': 12,
         '000101': 13,
         '011000': 13})

## sを求める

In [8]:
def get_s(n, oracle):
    r = list(oracle(Circuit(n*2).h[:n]).h[:n].m[:n].run(shots=100))
    A = [[int(r[i][j]) for j in range(n)] for i in range(n)]
    b = [0 for i in range(n)]

    f,r,x = gauss_elimination(A,b)
    s = 0
    for d in x:
        s <<= 1
        s += d
    return s

### (Z, xor, *)上のガウスの消去法
O(n^3)

In [9]:
def gauss_elimination(A, b):
    n = len(A)
    m = len(A[0])
    rank = 0
    cj = 0
    while rank < n and cj < m:
        for i in range(rank+1, n):
            if A[i][cj] > A[rank][cj]:
                A[i], A[rank] = A[rank], A[i]
                b[i], b[rank] = b[rank], b[i]
        if A[rank][cj]:
            d = A[rank][cj] ^ 1
            for j in range(m):
                A[rank][j] ^= d
            b[rank] ^= d
            for i in range(rank+1, n):
                k = A[i][cj]
                for j in range(m):
                    A[i][j] ^= k*A[rank][j]
                b[i] ^= k*b[rank]
            rank += 1
        cj += 1
    for i in range(rank, n):
        if b[i]:
            return False, rank, None
    if rank < m or cj < m:
        ci = rank
        for i in range(min(n, m)):
            if A[i][i] == 0:
                if i != ci:
                    A[i], A[ci] = A[ci], A[i]
                    b[i], b[ci] = b[ci], b[i]
                ci += 1
                for j in range(m):
                    A[i][j] = 0
                A[i][i] = 1
                b[i] = 1
    for j in range(m-1, -1, -1):
        for i in range(j):
            b[i] ^= b[j] * A[i][j]
    return True, rank, b

In [10]:
print(get_s(2, n2s1))
print(get_s(2, n2s2))
print(get_s(2, n2s3))
print(get_s(3, n3s3))

1
2
3
5
